### OLD
Now, let's doublecheck that those numbers make sense.  Make a plot of the HRD around the desired ($T_{\rm eff}$, $L_\ast$\) point, and plot the mass tracks and isochrones that neighbor the derived values:

In [ ]:
#mass tracks
Tlim = [5000, 4000]    # note: Teff is plotted backwards (see note below)
Llim = [0.5, 2]
fig, ax = plt.subplots()
ax.plot(4376, 1.5, marker='o', markersize=10, color="red")
ax.set_xlim(Tlim)
ax.set_ylim(Llim)
ax.set_xlabel(r'$\log{T_{\rm eff}} \,\, [{\rm K}]$')
ax.set_ylabel(r'$\log{L_\ast} \,\, [{\rm L}_\odot]$')

track_num = [80, 90, 100, 110, 120]

for i in track_num: #mass tracks near 0.93 Msun
    file_track_mass = working_dir+'MIST/'+filename_eep+'/%sM.track.eep'% ("{:05d}".format(i))
    track_mass = ascii.read(file_track_mass)
    age, logL, logTe = track_mass['col1'], track_mass['col7'], track_mass['col12']
    ax.plot(10**logTe[age < 1e8], 10**logL[age < 1e8], label = 'MT %s M$\odot$' % (i/100))
    ax.set_xlim(Tlim)
    ax.set_ylim(Llim)
    ax.set_xlabel(r'${T_{\rm eff}} \,\, [{\rm K}]$')
    ax.set_ylabel(r'${L_\ast} \,\, [{\rm L}_\odot]$')

#isochrone
log_age = [np.log10(1000000), np.log10(2000000)] #1Myr and 2Myr
colors = ['C1', 'C2']

for i in range(len(log_age)):
    close_age = iso_AGE[np.abs(iso_AGE - log_age[i]).argmin()] #np.searchsorted(iso_AGE, i)] gives the idx+1
    iso_L_Myr = iso_L[iso_AGE == close_age]
    iso_T_Myr = iso_T[iso_AGE == close_age]
    ax.plot(10**iso_T_Myr, 10**iso_L_Myr, ':'+colors[i], label= "Iso %d Myr" % (round(10**log_age[i])/1e6))
    ax.legend()

Ok, now we can also look into the weirdness that happens in the previous plot at $T_{\rm eff} \gtrsim 4500$ K.  I think this is not some bad interpolation issue, but rather just that this is where the mass tracks start to get complicated (bend over to higher $T_{\rm eff}$ at nearly fixed $L_\ast$) in the HRD.  Examine this issue by plotting the relevant part of the HRD and the mass tracks in that part of parameter-space.  (It might help to overlay the fixed $L_\ast$ slice we're considering as a horizontal line.). What's happening?  Are the mass tracks double-valued in $T_{\rm eff}$ at this $L_\ast$?  

In [ ]:
Tlim = [7000, 4000]    # note: Teff is plotted backwards (see note below)
Llim = [0.5, 3]
fig, ax = plt.subplots()
x = np.linspace(4000, 7000, 1000)
y = np.linspace(1.5, 1.5, 1000)
ax.plot(x, y, ':r', label=r'fixed 1.5 $L_\ast$')

track_num = [100, 104, 108, 112, 116, 120, 124, 128]

for i in track_num: #mass tracks near 0.93 Msun
    file_track_mass = working_dir+'MIST/'+filename_eep+'/%sM.track.eep'% ("{:05d}".format(i))
    track_mass = ascii.read(file_track_mass)
    age, logL, logTe = track_mass['col1'], track_mass['col7'], track_mass['col12']
    ax.plot(10**logTe[age < 1e8], 10**logL[age < 1e8], label = 'MT %s M$\odot$' % (i/100))
    ax.set_xlim(Tlim)
    ax.set_ylim(Llim)
    ax.set_xlabel(r'${T_{\rm eff}} \,\, [{\rm K}]$')
    ax.set_ylabel(r'${L_\ast} \,\, [{\rm L}_\odot]$')
    ax.legend()

Mass track of 1.2 M$\odot$ has overlapping value at 1.5 $L_\ast$

### attempt to generate image plots

In [ ]:
# generate spaced values (log scale)
tspce = np.linspace(np.log10(4000), np.log10(7000), 1000)
lspce = np.linspace(np.log10(0.1), np.log10(100), 1000)

# generate grid and coordinates
xy, yx = np.meshgrid(tspce, lspce) #grid
lumteffgrid = np.column_stack((xy.ravel(), yx.ravel())) 
#lumteffgrid = np.dstack((xy.T, yx.T)) #coordinates starting from (tspace[0], lspace[0])

# the input points in the HRD
points = np.column_stack((logTe, logL))

# MASS
# the values of mass corresponding to those points
values = logM

mass_est = 10**griddata(points, values, lumteffgrid, method = 'linear')
# test = griddata(points, values, (tspce[0], lspce[0]), method = 'linear')
# test2 = griddata(points, values, (tspce[-1], lspce[0]), method = 'linear')

mass_image = np.reshape(mass_est, (1000, 1000))

# mass_image = np.rot90(mass_est)
# #if you want to plot high to low teff, then flip mass_image about axis 1
# mass_image = np.flip(mass_image, axis=1)

In [ ]:
# Mass Plot
fig, ax = plt.subplots()
im = ax.imshow(mass_image, origin='lower', aspect='auto', extent=[4000,7000,0.1,100])
ax.set_xlabel(r'$T_{\rm eff} \,\, [{\rm K}]$', labelpad=10)
ax.set_ylabel(r'${L_\ast} \,\, [{\rm L}_\odot]$', labelpad=10)

cb = plt.colorbar(im, ax=ax, pad = 0.05)
cb.set_label(r'${M_\ast} \,\, [{\rm M}_\odot]$', rotation=270, labelpad=20)

track_num = [100, 200, 300, 400, 500]

Tlim = [4000, 7000]    # note: Teff is plotted backwards (see note below)
Llim = [0.1, 100]

for i in track_num: #mass tracks 1-5 solar mass
    file_track_mass = working_dir+'MIST/'+filename_eep+'/%sM.track.eep'% ("{:05d}".format(i))
    track_mass = ascii.read(file_track_mass)
    age, logL, logTe = track_mass['col1'], track_mass['col7'], track_mass['col12']
    ax.plot(10**logTe[age < 1e8], 10**logL[age < 1e8], label = 'MT %s M$\odot$' % (i/100))
    ax.set_xlim(Tlim)
    ax.set_ylim(Llim)
    ax.legend()

In [ ]:
# AGES
# the values of age corresponding to those points
values = logt
age_est = 10**(griddata(points, values, lumteffgrid, method= 'linear'))/1e6

age_image = np.rot90(age_est)
#if you want to plot high to low teff, then flip mass_image about axis 1
age_image = np.flip(age_image, axis=1) 
    
# Ages Plot
fig, ax = plt.subplots()
im = ax.imshow(age_image, origin='lower', aspect='auto', extent=[7000,4000,0.1,100])
ax.set_xlabel(r'$T_{\rm eff} \,\, [{\rm K}]$', labelpad=10)
ax.set_ylabel(r'${L_\ast} \,\, [{\rm L}_\odot]$', labelpad=10)

cb = plt.colorbar(im, ax=ax, pad = 0.05)
cb.set_label('age [Myr]', rotation=270, labelpad=20)

log_age = [np.log10(1e7), np.log10(2e7), np.log10(4e7), np.log10(8e7) ] #1Myr and 2Myr
colors = ['C1', 'C2', 'C3', 'C4']

Tlim = [7000, 4000]    # note: Teff is plotted backwards (see note below)
Llim = [0.1, 100]

for i in range(len(log_age)):
    close_age = iso_AGE[np.abs(iso_AGE - log_age[i]).argmin()] #np.searchsorted(iso_AGE, i)] gives the idx+1
    iso_L_Myr = iso_L[iso_AGE == close_age]
    iso_T_Myr = iso_T[iso_AGE == close_age]
    ax.plot(10**iso_T_Myr, 10**iso_L_Myr, ':'+colors[i], label= "Iso %d Myr" % (round(10**log_age[i])/1e6))
    ax.set_xlim(Tlim)
    ax.set_ylim(Llim)
    ax.legend()
